In [10]:
import pickle
import numpy as np
# from ExtractBootlegFeatures import *
from ExtractTest import *
import itertools
import numba as nb
from numba import jit
from collections import defaultdict

In [11]:
import dill
from glob import iglob

In [5]:
db_dir = '/data1/kji/databases2'

In [6]:
files = []
for filename in iglob(f"{db_dir}/*.pkl", recursive=True):
    files.append(filename)
files.sort(key = len)

In [7]:
with open("/data1/kji/databases2/160mill.pkl", "rb") as f:
    database = pickle.load(f)

In [8]:
def add_db_values(file_list, db):
    for file in file_list:
        with open(file, "rb") as f:
            d = dill.load(f)
        for n_gram in d.keys():
            if n_gram in db:
                if db[n_gram] is None:
                    db[n_gram] = d[n_gram]
                else:
                    db[n_gram].update(d[n_gram])
        d.clear()
        print(f"finished {file}")

In [13]:
add_db_values(files[:16], database)

finished /data1/kji/databases2/0.pkl
finished /data1/kji/databases2/02.pkl
finished /data1/kji/databases2/03.pkl
finished /data1/kji/databases2/05.pkl
finished /data1/kji/databases2/04.pkl
finished /data1/kji/databases2/01.pkl
finished /data1/kji/databases2/015.pkl
finished /data1/kji/databases2/014.pkl
finished /data1/kji/databases2/035.pkl
finished /data1/kji/databases2/013.pkl
finished /data1/kji/databases2/023.pkl
finished /data1/kji/databases2/025.pkl
finished /data1/kji/databases2/045.pkl
finished /data1/kji/databases2/034.pkl
finished /data1/kji/databases2/012.pkl
finished /data1/kji/databases2/024.pkl


In [17]:
counts = {}
for fp in database:
    counts[fp] = sum([len(database[fp][piece]) for piece in database[fp]])

In [12]:
with open("/data1/kji/databases/split21_Bt_200mill.pkl", "rb") as f:
    d = pickle.load(f)

In [18]:
with open("/data1/kji/databases/probabilities.pkl", "rb") as f:
    utilities = pickle.load(f)

In [19]:
with open("num_to_piece.pkl", 'rb') as f:
    num_to_piece = pickle.load(f)

In [39]:
len(num_to_piece)

32234

In [ ]:
with open("/data1/kji/databases/split21_Bt_200mill_counts.pkl", "rb") as f:
    counts = pickle.load(f)

50k budget per store
budget per aisle = 50k/len(bscore_query.T)

In [20]:
combinations = []
for n_gram in range(1, 5):
    combinations += [[0] + list(tup) for tup in itertools.combinations(range(1, 6), n_gram-1)]

In [21]:
combinations = ["".join(str(num) for num in combination) for combination in combinations]

In [22]:
powers = 1 << np.arange(62)[::-1]

In [23]:
def compute_fingerprint(cols):
    fp = []
    equals_Zero = True
    for column in cols:
        hashint = int(column.dot(powers))
        fp.append(hashint)
        if hashint != 0:
            equals_Zero = False
    if equals_Zero == True:
        return None
    return tuple(fp)

In [24]:
def utility(combination, matches):
    return utilities[combination] / matches

In [210]:
def get_ratios(bscore_query, counts_dict):
    l = len(bscore_query)
    # ratios[i][j] is a pair of (ratio, fingerprint)
    ratios = np.array([[(0, None) for _ in range(l)] for _ in range(26)])
    for j in range(l):
        # calculate utility to cost ratio for all 26 n-grams
        for idx, combination in enumerate(combinations):
            cols = []
            # we need at least enough fingerprints for all the indices in our combination
            try:
                for i in combination:
                    cols.append(bscore_query[j+int(i)])
            except IndexError:
                continue
            fp = compute_fingerprint(cols)
            if not fp:
                continue
            matches = counts_dict.get(fp, 0)
            if matches == 0:
                continue
            ratios[idx][j] = (utility(combination, matches), fp)
    return ratios

In [197]:
with open("bootleg_total/p111.pkl", "rb") as f:
    data = pickle.load(f)
data = data.T
fps = []
for colindex in range(len(data)):
    for combination in combinations:
        cols = []
        # we need at least enough fingerprints for all the indices in our combination
        try:
            for i in combination:
                cols.append(data[colindex+int(i)])
        except IndexError:
            continue
        fp = []
        equals_Zero = True
        for column in cols:
            hashint = int(column.dot(powers))
            fp.append(hashint)
            if hashint != 0:
                equals_Zero = False
        if equals_Zero == True:
            continue
        fps.append(tuple(fp))

In [198]:
fps = set(fps)

In [154]:
def get_fingerprints(bscore_query, rindex_dict, ratios, runtime_budget):
    l = len(bscore_query)
    aisle_budget = runtime_budget / l
    cur_budget = aisle_budget
    offset_dict = defaultdict(list)
    for i in range(l):
        fingerprints = []
        col = ratios[:, i]
        num_lookups = np.count_nonzero(col)
        lookups = sorted(col, key = lambda x: x[0], reverse = True)
        for _, n_gram in lookups:
            if not n_gram or cur_budget < 0:
                break
            fingerprints.append(n_gram)
            cur_budget -= 1
        cur_budget += aisle_budget
        for fp in fingerprints:
            pieces_and_offsets = rindex_dict[fp]
            for piece in pieces_and_offsets:
                offset = [(j - i, num_lookups) for j in pieces_and_offsets[piece]]
                offset_dict[num_to_piece[piece]].extend(offset)
    return offset_dict

In [217]:
def rankHistograms(offset_dict, bin_size=10):
    histograms = {}
    bin_size = 1
    pieceScores = []
    numShow = 5
    for key in offset_dict:
        h = offset_dict[key]
        maxh = max(h)[0]
        minh = min(h)[0]
        if(maxh > minh+bin_size):
            hist = np.zeros(int((maxh-minh)/bin_size)+2)
            for i in h:
                # i[1] is the number of lookups we did for that column
                hist[int((i[0]-minh)/bin_size)] += 1/i[1]
            score = np.max(hist)
            pieceScores.append((key, score))
            
    pieceScores = sorted(pieceScores, key = lambda x:x[1], reverse=True)
    return pieceScores, histograms

In [209]:
def processSingleQuery(imagefile, rindex, counts, runtime_budget, outfile = None):
    profileStart = time.time()
    
    # Get Bootleg Score
    bscore_query = processQuery(imagefile)
    bscore_query = bscore_query.T
    
    searchStart = time.time()
    # Generate and rank histograms
    
    ratios = get_ratios(bscore_query, counts)
    offset_dict = get_fingerprints(bscore_query, rindex, ratios, runtime_budget)
    pieceScores, histograms = rankHistograms(offset_dict)
    
    # Profile & save to file
    profileEnd = time.time()
    
    profileDur = profileEnd - profileStart
    print(profileDur)
    saveToFile(outfile, imagefile, pieceScores, profileDur)
    return pieceScores

In [206]:
%load_ext line_profiler

In [216]:
%lprun -f processSingleQuery processSingleQuery('data/queries/p1_q2.jpg', database, counts, 1000)

Processing data/queries/p1_q2.jpg
1.009040117263794


In [29]:
def saveToFile(outfile, imagefile, pieceScores, profileDur):
    if outfile:
        with open(outfile, 'wb') as f:
            query = os.path.splitext(os.path.basename(imagefile))[0]
            pickle.dump((query,pieceScores, profileDur),f)

In [30]:
def processQuery_wrapper(queryfile, rindex, counts, outdir, runtime_budget):
    # wrapper for running multiple jobs in parallel
    basename = os.path.splitext(os.path.basename(queryfile))[0] # e.g. p1_q1
    hyp_outfile = "{}/{}.hyp".format(outdir, basename)
    piece = basename.split('_')[0]
    # might change later to print to outfile
    return processSingleQuery(queryfile, rindex, counts, runtime_budget, hyp_outfile)

In [58]:
runtime_budget = 100000

In [71]:
query_list = 'cfg_files/query.train.list' # list of query images
outdir = 'experiments/160mill_other/hyp' # where to save hypothesis output files

# prep output directory
if not os.path.isdir(outdir):
    os.makedirs(outdir)

# load reverse index. Recommend keeping load=False and loading it earlier.
load = False
if load:
    print("LOADING RINDEX")
    rindex1 = []
    with open(pickle_file, 'rb') as f:
        rindex1 = pickle.load(f)
    rindex_filter = rindex1
    
print("STARTING PROCESSING")
# number of cores to use
multiprocess = False
if multiprocess:
    n_cores = 25 #multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=n_cores)

inputs = []
with open(query_list, 'r') as f:
    for line in f:
        inputs.append((line.rstrip(), outdir))
        
if multiprocess:
    # process queries in parallel
    outputs = list(pool.starmap(processQuery_wrapper, inputs))
else:
    for i in inputs:
        processQuery_wrapper(i[0], database, counts, i[1], runtime_budget)

STARTING PROCESSING
Processing data/queries/p1_q1.jpg
1.1950154304504395
Processing data/queries/p1_q2.jpg
0.9447422027587891
Processing data/queries/p1_q3.jpg
0.9047274589538574
Processing data/queries/p1_q4.jpg
0.8947396278381348
Processing data/queries/p1_q5.jpg
0.9015076160430908
Processing data/queries/p1_q6.jpg
0.8993818759918213
Processing data/queries/p1_q7.jpg
0.9640214443206787
Processing data/queries/p1_q8.jpg
0.9908359050750732
Processing data/queries/p1_q9.jpg
0.8765914440155029
Processing data/queries/p1_q10.jpg
0.9551253318786621
Processing data/queries/p5_q1.jpg
0.8657801151275635
Processing data/queries/p5_q2.jpg
0.9099273681640625
Processing data/queries/p5_q3.jpg
1.020650863647461
Processing data/queries/p5_q4.jpg
0.9755415916442871
Processing data/queries/p5_q5.jpg
1.0789053440093994
Processing data/queries/p5_q6.jpg
0.9937808513641357
Processing data/queries/p5_q7.jpg
1.0858666896820068
Processing data/queries/p5_q8.jpg
1.0377423763275146
Processing data/queries/p5

1.0340182781219482
Processing data/queries/p75_q4.jpg
1.0001463890075684
Processing data/queries/p75_q5.jpg
1.1777012348175049
Processing data/queries/p75_q6.jpg
1.1608710289001465
Processing data/queries/p75_q7.jpg
1.162827730178833
Processing data/queries/p75_q8.jpg
1.1996171474456787
Processing data/queries/p75_q9.jpg
1.1594271659851074
Processing data/queries/p75_q10.jpg
1.1064379215240479
Processing data/queries/p81_q1.jpg
1.1004559993743896
Processing data/queries/p81_q2.jpg
0.9801290035247803
Processing data/queries/p81_q3.jpg
1.0939154624938965
Processing data/queries/p81_q4.jpg
1.074542760848999
Processing data/queries/p81_q5.jpg
0.8998842239379883
Processing data/queries/p81_q6.jpg
0.9765322208404541
Processing data/queries/p81_q7.jpg
0.990851879119873
Processing data/queries/p81_q8.jpg
1.00738525390625
Processing data/queries/p81_q9.jpg
1.0284039974212646
Processing data/queries/p81_q10.jpg
0.828392505645752
Processing data/queries/p85_q1.jpg
0.9257426261901855
Processing da

0.8856511116027832
Processing data/queries/p151_q5.jpg
1.0005171298980713
Processing data/queries/p151_q6.jpg
1.021916389465332
Processing data/queries/p151_q7.jpg
0.9542446136474609
Processing data/queries/p151_q8.jpg
1.042769193649292
Processing data/queries/p151_q9.jpg
0.9434075355529785
Processing data/queries/p151_q10.jpg
1.0354206562042236
Processing data/queries/p155_q1.jpg
1.1477656364440918
Processing data/queries/p155_q2.jpg
0.9922647476196289
Processing data/queries/p155_q3.jpg
1.2420697212219238
Processing data/queries/p155_q4.jpg
1.0118632316589355
Processing data/queries/p155_q5.jpg
1.0105583667755127
Processing data/queries/p155_q6.jpg
1.032728910446167
Processing data/queries/p155_q7.jpg
0.36345982551574707
Processing data/queries/p155_q8.jpg
1.0041217803955078
Processing data/queries/p155_q9.jpg
1.0604424476623535
Processing data/queries/p155_q10.jpg
1.0553245544433594
Processing data/queries/p161_q1.jpg
0.72464919090271
Processing data/queries/p161_q2.jpg
0.8783686161

In [53]:
len(database)

138390672

In [202]:
n_gram_count = defaultdict(int)

In [203]:
for i in database:
    n_gram_count[len(i)] += 1

In [204]:
n_gram_count

defaultdict(int, {1: 380992, 2: 28502993, 3: 109506687})

In [54]:
sum(counts.values())

249398153

In [121]:
query_list = 'cfg_files/query.train.list' # list of query images
outdir = 'experiments/160mill_real/hyp' # where to save hypothesis output files

# prep output directory
if not os.path.isdir(outdir):
    os.makedirs(outdir)

# load reverse index. Recommend keeping load=False and loading it earlier.
load = False
if load:
    print("LOADING RINDEX")
    rindex1 = []
    with open(pickle_file, 'rb') as f:
        rindex1 = pickle.load(f)
    rindex_filter = rindex1
    
print("STARTING PROCESSING")
# number of cores to use
multiprocess = False
if multiprocess:
    n_cores = 25 #multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=n_cores)

inputs = []
with open(query_list, 'r') as f:
    for line in f:
        inputs.append((line.rstrip(), outdir))
        
if multiprocess:
    # process queries in parallel
    outputs = list(pool.starmap(processQuery_wrapper, inputs))
else:
    for i in inputs:
        processQuery_wrapper(i[0], database, counts, i[1], runtime_budget)

STARTING PROCESSING
Processing data/queries/p1_q1.jpg
1.0397229194641113
Processing data/queries/p1_q2.jpg
0.8993687629699707
Processing data/queries/p1_q3.jpg
0.8684978485107422
Processing data/queries/p1_q4.jpg
0.9074606895446777
Processing data/queries/p1_q5.jpg
0.8628940582275391
Processing data/queries/p1_q6.jpg
0.91115403175354
Processing data/queries/p1_q7.jpg
0.92041015625
Processing data/queries/p1_q8.jpg
0.8913085460662842
Processing data/queries/p1_q9.jpg
0.7612733840942383
Processing data/queries/p1_q10.jpg
0.9205169677734375
Processing data/queries/p5_q1.jpg
0.8310346603393555
Processing data/queries/p5_q2.jpg
0.8689148426055908
Processing data/queries/p5_q3.jpg
0.8901717662811279
Processing data/queries/p5_q4.jpg
0.9222903251647949
Processing data/queries/p5_q5.jpg
0.9390726089477539
Processing data/queries/p5_q6.jpg
0.88665771484375
Processing data/queries/p5_q7.jpg
0.9725544452667236
Processing data/queries/p5_q8.jpg
0.8942029476165771
Processing data/queries/p5_q9.jpg


1.0944933891296387
Processing data/queries/p75_q4.jpg
0.8749768733978271
Processing data/queries/p75_q5.jpg
1.0666193962097168
Processing data/queries/p75_q6.jpg
1.0355610847473145
Processing data/queries/p75_q7.jpg
1.0805203914642334
Processing data/queries/p75_q8.jpg
1.0294954776763916
Processing data/queries/p75_q9.jpg
1.0902042388916016
Processing data/queries/p75_q10.jpg
1.0077955722808838
Processing data/queries/p81_q1.jpg
0.9768671989440918
Processing data/queries/p81_q2.jpg
0.8842289447784424
Processing data/queries/p81_q3.jpg
0.9750993251800537
Processing data/queries/p81_q4.jpg
0.9805829524993896
Processing data/queries/p81_q5.jpg
0.7985785007476807
Processing data/queries/p81_q6.jpg
0.8613741397857666
Processing data/queries/p81_q7.jpg
0.9249756336212158
Processing data/queries/p81_q8.jpg
0.8906087875366211
Processing data/queries/p81_q9.jpg
0.9585118293762207
Processing data/queries/p81_q10.jpg
0.7608540058135986
Processing data/queries/p85_q1.jpg
0.9051644802093506
Process

0.9636619091033936
Processing data/queries/p151_q4.jpg
0.8240842819213867
Processing data/queries/p151_q5.jpg
1.0345141887664795
Processing data/queries/p151_q6.jpg
0.9694733619689941
Processing data/queries/p151_q7.jpg
0.9115679264068604
Processing data/queries/p151_q8.jpg
0.9701948165893555
Processing data/queries/p151_q9.jpg
0.962486743927002
Processing data/queries/p151_q10.jpg
1.002488136291504
Processing data/queries/p155_q1.jpg
1.1372654438018799
Processing data/queries/p155_q2.jpg
0.9702911376953125
Processing data/queries/p155_q3.jpg
1.1122856140136719
Processing data/queries/p155_q4.jpg
1.0355579853057861
Processing data/queries/p155_q5.jpg
0.9326856136322021
Processing data/queries/p155_q6.jpg
0.941896915435791
Processing data/queries/p155_q7.jpg
0.290539026260376
Processing data/queries/p155_q8.jpg
0.9773919582366943
Processing data/queries/p155_q9.jpg
0.944098949432373
Processing data/queries/p155_q10.jpg
0.9421160221099854
Processing data/queries/p161_q1.jpg
0.68434667587

## Test code

In [62]:
piece_counts = defaultdict(int)
for fp in database:
    for piece in database[fp]:
        piece_counts[piece] += 1

In [63]:
def get_coverage(counts, total_pieces):
    """
    given a dictionary of counts, return an array where a[i] is the percent of 
    items in IMSLP with at least i+1 unique fingerprints
    """
    num_pieces_per_count = defaultdict(int)
    for num in counts.values():
        num_pieces_per_count[num] += 1
    n = max(num_pieces_per_count)
    cumulative_counts = []
    cur_count = 0
    for i in reversed(range(1, n+1)):
        cur_count += num_pieces_per_count[i]
        cumulative_counts.append(cur_count)
    return [count / total_pieces for count in cumulative_counts[::-1]]

In [64]:
total = len(num_to_piece)

In [67]:
percents = get_coverage(piece_counts, total)

In [69]:
with open("data/percent_140mill.pkl", "wb") as f:
    pickle.dump(percents, f)

In [76]:
from collections import Counter

In [83]:
total

32234

In [123]:
num_to_piece[32144]

'p111'

In [87]:
query_count = 0
for i in range(32034, 32234):
    if i in piece_counts:
        query_count += 1

In [194]:
max(piece_counts.values())

198965

In [195]:
piece_counts[32144]

768

In [82]:
len(piece_counts)

29539